# This Notebook Reads the PIRUS Codebook and Converts it into a Json

In [1]:
from collections import defaultdict
import sys
print(sys.version)
import numpy as np
print(np.__version__)
import pandas as pd
print(pd.__version__)
import re
import json

3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ]
1.19.2
1.1.3


In [2]:
codebook = open(r"../og_data/PIRUS_May2020/pirus_codebook.txt","r")
cb = codebook.read()

cb = cb.replace('  ','')

cb = cb.split('''Management and Identification Variables''')[2]
cb = cb.split('\n')
cb = list(filter(lambda x: x != "" and len(x) != 1, cb))

new_cb = []
for c in cb:
    try:
        x = int(c)
    except:
        x = 0
    if x == 0:
        new_cb.append(c)
cb = '\n'.join(new_cb)
pattern = re.compile('\d+. Field Name: ')
data = re.split(pattern, cb)[1:]

In [3]:
codebook = defaultdict(dict)
keys = ['Label(s)', 'Type', 'Source', 'Description', 'Note']
note_count = 0
for entry in data:
    # Isolate Field Name
    pattern = re.compile('(\S+)')
    field = re.search(pattern, entry).group()
    codebook[field.lower()].update({'name' : field})
    entry = entry.replace(field, '') # remove field from entry

    for i in range(len(keys)-1,0,-1):
        if keys[i] in entry:
            k = keys[i]
            pattern = re.compile(r'' + k + ': (.+)\n')
            d = re.search(pattern, entry).group(1)
            value = d[0].upper() + d[1:].strip()
            entry = entry.replace(k + ': ' + d, '')# remove note from entry
        else:
            value = ''  
        codebook[field.lower()].update({keys[i].lower() : value})
    codebook[field.lower()].update({'rest_text' : entry})

In [4]:
# translate codes for field
for k in codebook.keys():
    codes = {}
    entries = codebook[k]['rest_text'].split('\n')
    entries = list(filter(lambda x: x != "", entries))

    # Find the first instance of ' = ' in e in entry, 
    for e in range(len(entries)):
        if ' = ' in entries[e]:
            break
    beginning_entries = entries[:e]
    entries = entries[e:]
    
    # Find the last instance of ' = ' in e in entry, 
    for e in range(len(entries)-1,-1,-1):
        if ' = ' in entries[e]:
            break
    end_entries = entries[e + 1:]
    entries = ' '.join(entries[:e + 1])

    pattern = re.compile('-?\d+\s=')
    lp = re.findall(pattern, entries)

    for l in range(len(lp)-1,-1, -1):

        key = int(lp[l][:-2])
        val = entries.split(lp[l])[1]
        val = val[0].upper() + val[1:]
        val = val.strip()
        codes[key] = val
        entries = entries.split(lp[l])[0]
        
    codebook[k]['codes'] = codes
    
    rest_text = ' '.join(beginning_entries + [entries] + end_entries).strip()
    codebook[k]['rest_text'] = rest_text

In [5]:
# print dictionary to json
with open('../clean_data/pirus_codebook.json','w+') as f:
    json.dump(codebook, f)